In [ ]:
import os, json, copy
import pandas as pd
from tqdm import tqdm

### 대화_데이터셋

In [ ]:
path_2 = '주제별 텍스트 일상 대화 데이터/'
file_list_2 = os.listdir(path_2)


li = []
for i in tqdm(range(len(file_list_2))):
    folder = file_list_2[i]

    path_1 = '주제별 텍스트 일상 대화 데이터/'+folder+'/'
    file_list_1 = os.listdir(path_1)

    for n in range(len(file_list_1)):
        with open(path_1+file_list_1[n], 'r', encoding='utf-8') as file:
            temp_data = json.load(file)
            li.append(temp_data)

100%|██████████| 6/6 [00:07<00:00,  1.17s/it]


In [ ]:
def preprocessing_list(x):
    dial_list = x['info'][0]['annotations']['text'].split('\n')
    temp_list = []; cnt = 0
    temp_list = copy.deepcopy(dial_list)
    for i in range(len(dial_list)):
        i -= cnt
        if temp_list[i][0] not in ['1','2','3','4','5'] :
            temp_list[i-1] += temp_list[i].lstrip()
            temp_list.pop(i); cnt += 1
    return temp_list

In [ ]:
dialog = pd.DataFrame(columns=['Q','Q_intent', 'A', 'A_intent'])

for i in tqdm(range(len(li))):
    sen = preprocessing_list(li[i])
    intent = li[i]['info'][0]['annotations']['lines']
    for j in range(len(sen)):
        if len(sen) != len(intent):
            print(f"len(sen) != len(intent) : {li[i]['dataset']['name']}")
            break
        else :
            for k in range(1,6):
                if j+k >= len(sen)-1:
                    break
                elif sen[j][0] == sen[j+k][0]:
                    break

                if intent[j]['speechAct'] != intent[j+k]['speechAct']:
                    # dialog = pd.concat([dialog,pd.DataFrame({'Q' : [sen[j].split(':')[-1].lstrip()], 'Q_intent' : intent[j]['speechAct'], 'A' : [sen[j+k].split(':')[-1].lstrip()], 'A_intent' : intent[j+k]['speechAct']})],ignore_index=True)
                    try:
                        dialog = pd.concat([dialog,pd.DataFrame({'Q' : [sen[j].split(':')[-1].lstrip()],
                                                                 'Q_intent' : intent[j]['speechAct'],
                                                                 'A' : [sen[j+k].split(':')[-1].lstrip()],
                                                                 'A_intent' : intent[j+k]['speechAct']})], ignore_index=True)
                    except :
                        print(li[i]['dataset']['name'])
                        print(f'대화번호i:{i} / 문장번호j:{j} / 쌍번호k:{k}')
dialog

 13%|█▎        | 3508/26215 [00:51<07:02, 53.72it/s] 

KAKAO_1174_20_set
대화번호i:3499 / 문장번호j:22 / 쌍번호k:1


 15%|█▍        | 3876/26215 [01:06<29:50, 12.48it/s]

In [ ]:
dialog.to_csv('dataset/dialog_chatbot_source.csv')

In [ ]:
temp_df = pd.read_csv('dataset/dialog_chatbot_source.csv')
temp_df.drop(['Unnamed: 0'], axis=1, inplace=True)
temp_df.dropna(inplace=True)
temp_df.to_csv('dataset/dialog_chatbot.csv')

### 감성분류/의도분류 데이터셋

In [ ]:
dialog = pd.read_csv('dataset/dialog_chatbot.csv')

In [ ]:
intent_model = pd.DataFrame(columns=['sentence','intent'])
Q_temp_sen = pd.DataFrame()
Q_temp_int = pd.DataFrame()
Q_temp_sen['sentence'] = dialog['Q']
Q_temp_int['intent'] = dialog['Q_intent']
Q_temp = pd.concat([Q_temp_sen,Q_temp_int], axis=1)

A_temp_sen = pd.DataFrame()
A_temp_int = pd.DataFrame()
A_temp_sen['sentence'] = dialog['A']
A_temp_int['intent'] = dialog['A_intent']
A_temp = pd.concat([A_temp_sen,A_temp_int], axis=1)

intent_model = pd.concat([Q_temp,A_temp], ignore_index=True)
intent_model.drop(intent_model.loc[intent_model['intent']=='N/A'].index, axis=0, inplace=True)
intent_model

,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
3,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
4,하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,(단언) 진술하기
...,...,...
2211833,매매로 하시면 주택을 선호하시나요 아파트를 선호하시나요?,(지시) 질문하기
2211834,처음에는 아파트로 시작하는 게 좋을 거 같아요.,(표현) 사과하기
2211835,주택 청약을 노리고 있긴 한데 요즘에는 로또 청약이라고 해서 당첨되기가 하늘의 별 ...,(단언) 진술하기
2211836,저도 그런 거에 관심을 좀 가져봐야겠어요 ㅠ,(단언) 주장하기


In [ ]:
intent_model.to_csv('dataset/temp.csv')

In [ ]:
df = pd.read_csv('dataset/temp.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)    # 불필요 column 제거
df.dropna(inplace=True)    # 결측치 제거
# df.drop(df[df.duplicated()])
df.drop(df[df.duplicated()].index, axis=0, inplace=True)    # 중복값 제거
df.drop(df[df['intent'] == '(언약) 위협하기'].index, axis=0, inplace=True)    # 불필요 label 제거
df.reset_index(inplace=True)    # 인덱스 초기화
df.drop(['index'], axis=1, inplace=True)    # 불필요 column 제거

In [ ]:
df

,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
3,하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,(단언) 진술하기
4,아닌데 그냥 많이 먹은 거 같은데,(단언) 주장하기
...,...,...
1159979,일단 전세로 생각하고 있습니다,(단언) 주장하기
1159980,전세로 살다가 형편 좀 괜찮아지면 집을 사는 걸로 생각 중입니다,(단언) 주장하기
1159981,요즘 전세 가격도 만만치 않다고 하더라고요.,(단언) 주장하기
1159982,처음에는 아파트로 시작하는 게 좋을 거 같아요.,(단언) 진술하기


In [ ]:
df.to_csv('intent_model_dataset.csv')

In [ ]:
df

,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
3,하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,(단언) 진술하기
4,아닌데 그냥 많이 먹은 거 같은데,(단언) 주장하기
...,...,...
1159979,일단 전세로 생각하고 있습니다,(단언) 주장하기
1159980,전세로 살다가 형편 좀 괜찮아지면 집을 사는 걸로 생각 중입니다,(단언) 주장하기
1159981,요즘 전세 가격도 만만치 않다고 하더라고요.,(단언) 주장하기
1159982,처음에는 아파트로 시작하는 게 좋을 거 같아요.,(단언) 진술하기
